### Conv types 

In [8]:
import torch
x = torch.rand(1,3,200,210)

#### Depth-wise Convolution 

![](./fig/depthwise.png)

In [2]:
from torch import nn
class depthwise_conv(nn.Module):
    def __init__(self, nin, kernels_per_layer):
        super(depthwise_separable_conv, self).__init__()
        self.depthwise = nn.Conv2d(nin, nin * kernels_per_layer, kernel_size=3, padding=1, groups=nin)


    def forward(self, x):
        out = self.depthwise(x)
        return out

In [38]:
conv = nn.Conv2d(3,6,kernel_size=3,padding=1,groups=3)

print('weight',conv.weight.shape)

weight torch.Size([6, 1, 3, 3])


In [39]:
conv(x).shape

torch.Size([1, 6, 200, 210])

#### Point wise Conv (1x1 conv) 

In [40]:

class pointwise_conv(nn.Module):
    def __init__(self, nin, nout):
        super(depthwise_separable_conv, self).__init__()
        self.pointwise = nn.Conv2d(nin, nout, kernel_size=1)

    def forward(self, x):
        out = self.pointwise(x)
        return out

#### Depth wise separable conv  

![](./fig/depth_wise_separable.png)

In [41]:
# depth-wise conv + 1x1 conv
class depthwise_separable_conv(nn.Module):
    def __init__(self, nin, kernels_per_layer, nout):
        super(depthwise_separable_conv, self).__init__()
        self.depthwise = nn.Conv2d(nin, nin * kernels_per_layer, kernel_size=3, padding=1, groups=nin)
        self.pointwise = nn.Conv2d(nin * kernels_per_layer, nout, kernel_size=1)

    def forward(self, x):
        out = self.depthwise(x)
        out = self.pointwise(out)
        return out

#### Dilated Conv

![](./fig/14.gif)

In [44]:
nn.Conv2d(
    in_channels:int,
    out_channels:int,
    kernel_size:Union[int, Tuple[int, int]],
    stride:Union[int, Tuple[int, int]]=1,
    padding:Union[str, int, Tuple[int, int]]=0,
    dilation:Union[int, Tuple[int, int]]=1,
    groups:int=1,
    bias:bool=True,
    padding_mode:str='zeros',
    device=None,
    dtype=None,
) -> None

### LKA (Large Kernel Attention) 

In [ ]:
class LKA(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.conv0 = nn.Conv2d(dim, dim, 5, padding=2, groups=dim)
        self.conv_spatial = nn.Conv2d(dim, dim, 7, stride=1, padding=9, groups=dim, dilation=3)
        self.conv1 = nn.Conv2d(dim, dim, 1)


    def forward(self, x):
        u = x.clone()        
        attn = self.conv0(x)
        attn = self.conv_spatial(attn)
        attn = self.conv1(attn)

        return u * attn